### 지오 코딩 : 문자로 된 주소를 위도와 경도로 변환

#### 카카오맵 API
- https://developers.kakao.com/ -> [내 애플리케이션] -> [애플리케이션 추가하기] -> [REST API 키]

### 중복된 주소 제거

In [5]:
library(tidyverse)

In [6]:
load("./04_pre_process/04_preprocess.rdata")

In [8]:
apt_juso <- apt_price %>% select(juso_jibun) %>%
    unique

In [9]:
apt_price %>% dim
apt_juso %>% dim
head(apt_juso, 3) %>% print

[1] 43390    13

[1] 5717    1

                                     juso_jibun
1       서울특별시 종로구 청운동 56-45 청운현대
2 서울특별시 종로구 사직동 9-1 광화문스페이스본
3 서울특별시 종로구 내수동 71 경희궁의아침2단지


### 
### 주소를 좌표로 변경

In [11]:
library(httr)
library(RJSONIO)
library(data.table)

In [12]:
add_list <- list()
cnt <- 0
kakao_key <- "061a64ce5679dc124cb5883f846531d7"



### ```GET(url, query)```

In [ ]:
lon_lat <- GET(url = 'https://dapi.kakao.com/v2/local/search/address.json',
                       # 서비스 URL
                       query = list(query = '서울특별시 송파구 거여동 거여5단지'),
                       # 질의
                       add_headers(Authorization = paste0("KakaoAK ", kakao_key)))

### ```content(as = )``` : GET한 파일을 원하는 형식으로 저장
- ```as``` : 원본(raw), 텍스트(text) / (parsed) / content

### ```fromJSON()``` : JSON을 R객체로 변환

In [13]:
lon_lat %>% content(as = 'text') %>% RJSONIO::fromJSON()

$documents
$documents[[1]]
$documents[[1]]$address
          address_name                 b_code                 h_code 
"서울 송파구 거여동 5"           "1171011300"           "1171053100" 
       main_address_no            mountain_yn     region_1depth_name 
                   "5"                    "N"                 "서울" 
    region_2depth_name   region_3depth_h_name     region_3depth_name 
              "송파구"              "거여1동"               "거여동" 
        sub_address_no                      x                      y 
                    ""     "127.142029142591"     "37.4990648506116" 

$documents[[1]]$address_name
[1] "서울 송파구 거여동 5"

$documents[[1]]$address_type
[1] "REGION_ADDR"

$documents[[1]]$road_address
              address_name              building_name 
"서울 송파구 성내천로 124"                         "" 
          main_building_no         region_1depth_name 
                     "124"                     "서울" 
        region_2depth_name         region_3depth_name 
                  "송파구"                   "거여동" 
                 road_name            sub_building_no 
                "성내천로"                         "" 
            underground_yn                          x 
                       "N"         "127.142029936425" 
                         y                    zone_no 
        "37.4990660209644"                    "05736" 

$documents[[1]]$x
[1] "127.142029142591"

$documents[[1]]$y
[1] "37.4990648506116"



$meta
$meta$is_end
[1] TRUE

$meta$pageable_count
[1] 1

$meta$total_count
[1] 1

### ```tryCatch({})``` : 오류가 발생하여도 반복문을 멈추지 않고 다음 반복으로 건너 뜀 (예외 처리)
- 일반적인 반복문: 중간에 오류발생으로 멈춤

In [15]:
inputs = list(1, 2, 3, 'four', 5, 6)  
for(input in inputs) {  
  print(paste(input, "의 로그값은 =>", log(input)))
}

[1] "1 의 로그값은 => 0"
[1] "2 의 로그값은 => 0.693147180559945"
[1] "3 의 로그값은 => 1.09861228866811"


ERROR: Error in log(input): non-numeric argument to mathematical function


- tryCatch()함수가 포함된 반복문: 중간에 오류 발생하여도 예외 처리하고 넘어감

In [16]:
for(input in inputs) {  
  tryCatch({ 
    print(paste(input, "의 로그값은 =>", log(input)))
  }, error = function(e) {cat("ERROR :",conditionMessage(e), "\n")} )}

[1] "1 의 로그값은 => 0"
[1] "2 의 로그값은 => 0.693147180559945"
[1] "3 의 로그값은 => 1.09861228866811"
ERROR : non-numeric argument to mathematical function 
[1] "5 의 로그값은 => 1.6094379124341"
[1] "6 의 로그값은 => 1.79175946922805"


### 
### 주소를 좌표로 변경

In [ ]:
for(i in 1:nrow(apt_juso)) { 
    
    tryCatch(
    {
        # 주소로 좌표값 요청
        lon_lat <- GET(url = 'https://dapi.kakao.com/v2/local/search/address.json',
                       # 서비스 URL
                       query = list(query = apt_juso[i,]),
                       # 질의
                       add_headers(Authorization = paste0("KakaoAK ", kakao_key)))
                       # 헤더
      
        # 위경도만 추출하여 저장
        coordxy <- lon_lat %>% content(as = 'text') %>% RJSONIO::fromJSON()
      
        # 반복횟수 카운팅
        cnt = cnt + 1
      
        # 주소, 경도, 위도 정보를 리스트로 저장
        add_list[[cnt]] <- data.table(apt_juso = apt_juso[i,], 
                                      coord_x = coordxy$documents[[1]]$x, 
                                      coord_y = coordxy$documents[[1]]$y)
        
        # 진행상황 알림 메시지
        message <- paste0("[", i,"/",nrow(apt_juso),"] 번째 (",
                          round(i / nrow(apt_juso) * 100, 2), 
                          " %) [", apt_juso[i,],
                          "] 지오코딩 중입니다: X= ", add_list[[cnt]]$coord_x, 
                          " / Y= ", add_list[[cnt]]$coord_y)
        cat(message, "\n\n")
      
        # 예외처리 구문 종료
        }, 
        error = function(e) {cat("ERROR :",conditionMessage(e), "\n")}
    )
}

### 
### 지오 코딩 결과 저장

- 리스트를 데이터프레임 변환

In [27]:
add_list %>% head(3)

apt_juso,coord_x,coord_y
<chr>,<chr>,<chr>
서울특별시 종로구 청운동 56-45 청운현대,126.967381797143,37.586205890259
apt_juso,coord_x,coord_y
<chr>,<chr>,<chr>
서울특별시 종로구 사직동 9-1 광화문스페이스본,126.968880867164,37.5744172415207
apt_juso,coord_x,coord_y
<chr>,<chr>,<chr>
서울특별시 종로구 내수동 71 경희궁의아침2단지,126.971284886893,37.5739639296009


In [30]:
juso_geocoding <- rbindlist(add_list)  
juso_geocoding %>% head(3) %>% print

                                        apt_juso          coord_x
1:       서울특별시 종로구 청운동 56-45 청운현대 126.967381797143
2: 서울특별시 종로구 사직동 9-1 광화문스페이스본 126.968880867164
3: 서울특별시 종로구 내수동 71 경희궁의아침2단지 126.971284886893
            coord_y
1:  37.586205890259
2: 37.5744172415207
3: 37.5739639296009


In [18]:
juso_geocoding$coord_x <- as.numeric(juso_geocoding$coord_x) 
juso_geocoding$coord_y <- as.numeric(juso_geocoding$coord_y)
juso_geocoding <- na.omit(juso_geocoding)

In [21]:
dir.create("./05_geocoding")  
save(juso_geocoding, file="./05_geocoding/05_juso_geocoding.rdata") 
write.csv(juso_geocoding, "./05_geocoding/05_juso_geocoding.csv")